# Storing a Graph in Neo4j

In [ ]:
#!pip install -r requirements.txt

In [ ]:
from graphtastic.database.neo4j import Neo4jConnect 

## Connecting to Neo4j

Please make sure to start the database (**Python DB**) we created in the *5.01_Setup_and_query_Neo4J.ipynb* notebook. Once the service has started, we can connect to our *localhost*.

In [ ]:
connection = Neo4jConnect('bolt://localhost:7687', 'admin', 'testpython')
print(connection)

## Preprocessing data

In [ ]:
import csv

with open('./data/reachability_250.txt', 'r') as txt:
    reader = csv.reader(txt, delimiter=' ')
    edges = [edge for edge in reader][6:]
    edges = [[edge[0], edge[1], int(edge[2])*-1] for edge in edges]

In [ ]:
with open('./data/reachability.csv', 'w', newline='') as c:
    writer = csv.writer(c)
    for edge in edges:
        writer.writerow(edge)

## Moving nodes, edges and properties to Neo4j

Our Cypher scripts will read data from csv files, which on a local installation of Neo4j should be in a specific folder that Neo4j can access. Follow the steps below to get the data into Neo4j:

1. In the main Neo4j application window (not the Browser window for the current database), click the ... next to our newly created database. 
2. From there, choose Open folder, then Import, to open the /import folder in your operating system’s file browser. 
3. Copy reachability.csv and reachability-meta.csv to this import folder.
4. Now that Neo4j can read the files, let’s write some Cypher to access their content. The below need to be run in Neo4J's command line interface:

In [ ]:
%%writefile cypher/read_reachability.cql
LOAD CSV WITH HEADERS from 'file:///reachability-meta.csv' as row
CREATE (city:City {
    node_id: row.node_id,
    name: row.name,
    population: row.metro_pop,
    latitude: row.latitude,
    longitude: row.longitude
})

In [ ]:
%%writefile cypher/load_reachability.cql
LOAD CSV from 'file:///reachability.csv' as row
MATCH (from:City {node_id: row[0]})
MATCH (to:City {node_id: row[1]})
MERGE (from)-[:AIR_TRAVEL {distance: row[2]}]-(to)

Now we will use Python to interface with the newly connected data model in Neo4J. 

In [ ]:

cypher = 'MATCH (n) RETURN count(n)'
result = connection.query(cypher).data()
connection.close()
print(result)

In [ ]:
edge_test = [57,0]

In [ ]:
cypher = f'MATCH (n:City {{node_id:{edge_test[0]}}})' \
          '-[r:AIR_TRAVEL]->' \
         f'(m:City {{node_id:{edge_test[1]}}}) ' \
          'RETURN n.name, m.name, r.travel_time'
print(cypher)

In [ ]:
result = connection.query(cypher).data()
print(result)
